In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import get_db_url
import wrangle


In [2]:
def get_zillow_data():
    filename = 'zillow_data.csv'

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else:
        df = pd.read_sql(
            '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips  FROM properties_2017 
            JOIN propertylandusetype USING (propertylandusetypeid) 
            WHERE propertylandusedesc = 'Single Family Residential'; 
            '''
            ,
            get_db_url('zillow')
        )

        df.to_csv(filename)

        return df

In [3]:
df = get_zillow_data()

In [4]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [5]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [6]:
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

 a lot of nulls throughout the dataset

In [7]:
df.dropna().shape

(2140235, 7)

In [8]:
df = df[df['yearbuilt'].notna()]

In [9]:
df = df[df['taxamount'].notna()]

In [10]:
df = df[df['calculatedfinishedsquarefeet'].notna()]

In [11]:
df = df[df['taxvaluedollarcnt'].notna()]

In [12]:
df.shape

(2140235, 7)

In [13]:
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [14]:
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

#write a for loop that will change the columns from floats to ints 
for column in df.columns:
    if df[column] == df.bathroomcnt:
        continue
    elif df[column].dtype == 'float64':
        df[column] = df[column].astype(np.int)

In [15]:
df = df.astype({'bedroomcnt':'int', 'calculatedfinishedsquarefeet':'int', 'taxvaluedollarcnt':'int', 'yearbuilt':'int','fips':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    int64  
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  int64  
 3   taxvaluedollarcnt             int64  
 4   yearbuilt                     int64  
 5   taxamount                     float64
 6   fips                          int64  
dtypes: float64(2), int64(5)
memory usage: 130.6 MB


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2140235.0,3.301419,0.932600,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2140235.0,2.240352,0.990549,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2140235.0,1863.193970,1221.754161,1.00,1258.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2140235.0,460641.625164,677157.635675,22.00,189166.00,328296.00,534606.00,90188462.00
yearbuilt,2140235.0,1960.967545,22.150563,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2140235.0,5616.711322,7814.562798,6.34,2540.85,4111.47,6411.93,1078101.87
fips,2140235.0,6048.309556,20.344910,6037.00,6037.00,6037.00,6059.00,6111.00


In [17]:
df[df['calculatedfinishedsquarefeet'] < 400]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
14,0,0.0,171,6920,1973,255.17,6037
15,0,0.0,203,14166,1960,163.79,6037
392,1,1.0,371,358040,1953,4047.24,6111
866,0,0.0,220,13840,1988,327.80,6037
2313,0,0.0,352,25464,1948,415.92,6037
...,...,...,...,...,...,...,...
2146189,0,0.0,330,16311,1925,297.36,6037
2148704,1,1.0,396,100903,1926,1513.09,6037
2151106,1,1.0,364,262581,1923,3895.26,6059
2151361,1,1.0,384,263965,1922,3291.26,6059


In [18]:
df = df[df['calculatedfinishedsquarefeet'] > 400]

In [19]:
# finds all zillow places that have both no rooms and no bathrooms
df.loc[df['bathroomcnt'].eq(0) & df['bedroomcnt'].eq(0)]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
11,0,0.0,1200,5328,1972,91.60,6037
2306,0,0.0,1776,79574,1989,1137.90,6037
2676,0,0.0,1057,344835,1942,4528.51,6037
3530,0,0.0,1281,187179,1920,2412.05,6037
3740,0,0.0,800,22469,1951,273.95,6037
...,...,...,...,...,...,...,...
2140196,0,0.0,2738,741817,1947,8854.56,6059
2142189,0,0.0,462,26004,1980,643.55,6037
2144648,0,0.0,936,33811,1961,536.25,6037
2146660,0,0.0,676,234580,1922,3013.57,6037


In [20]:
df.shape

(2137915, 7)

In [21]:
# drop all listings that have no bedroom AND no bathroom 
df = df.drop(df.loc[df['bathroomcnt'].eq(0) & df['bedroomcnt'].eq(0)].index)

In [22]:
df.shape

(2135908, 7)

In [23]:
wrangle.wrangle_zillow()
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
18,3,1.0,1244,169471,1950,2532.88,6037
19,3,2.0,1300,233266,1950,3110.99,6037
